# Multimodal

## Setup

### Imports

In [12]:
# Imports
import os
import numpy as np
import pandas as pd
import random
from tqdm.notebook import tqdm
from collections import OrderedDict
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
import torchmetrics
from src.utils import *
from src.models import MLPSTILClassifier, Attention1DSTILClassifier, Attention2DSTILClassifier, MLPSTILRegressor, Attention1DSTILRegressor, Attention2DSTILRegressor
from src.train import train_mm, evaluate, train_mm_stil

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Set seed & device

In [4]:
set_seed(42)      
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Extract text feats from reports

In [3]:
%%script false --no-raise-error
# load biobert model & tokenizer
tokenizer = AutoTokenizer.from_pretrained('dmis-lab/biobert-large-cased-v1.1-mnli')
lm = AutoModel.from_pretrained('dmis-lab/biobert-large-cased-v1.1-mnli')
data_dir = '/mnt/disks/ext/data/gdc/tcga/brca'
output_dir = 'data/report_feats'
extract_text_features(lm, tokenizer, data_dir)

## Task: Predict sTILs from WSIs & reports

### Set hparams

In [5]:
bsz = 16 # batch size for dataloaders
img_channels_in = 2048  # emb dim of wsi feats
text_channels_in = 1024 # emb dim of report feats
hidden_dim_mlp = 16 # hidden layer dim of mlp
num_classes = 10 # number of stil levels

### Load data

In [8]:
# Create the dataset
root_dir = './'
data_file = 'data/stils/data_stils.csv'
train_data = MMDataset(root_dir, data_file, 'train')
val_data = MMDataset(root_dir, data_file, 'val')
test_data = MMDataset(root_dir, data_file, 'test')

# Create the dataloaders
train_loader = DataLoader(train_data, batch_size=bsz, shuffle=True, num_workers=4, collate_fn=MMDataset.mm_collate_fn)
val_loader = DataLoader(val_data, batch_size=bsz, shuffle=False, num_workers=4, collate_fn=MMDataset.mm_collate_fn)
test_loader = DataLoader(test_data, batch_size=bsz, shuffle=False, num_workers=4, collate_fn=MMDataset.mm_collate_fn)

### Train & eval

In [15]:
# %%script false --no-raise-error
# init model (one of: MLPSTILClassifier, MLPSTILRegressor, Attention1DSTILClassifier, Attention1DSTILRegressor, Attention2DSTILClassifier, Attention2DSTILRegressor) 
model = Attention1DSTILRegressor(img_channels_in, text_channels_in)

# set training args
args = {'num_epochs': 40, 'resume_ckpt': 'model_ckpts/Attention1DSTILRegressor/ckpt_best.ckpt'}

# train model
model, trainer = train_mm_stil(model, train_loader, val_loader, args)

# evaluate the trained model on the test set
trainer.test(model, test_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/conda/envs/multimodal/lib/python3.11/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:615: UserWarning: Checkpoint directory /home/neil/multimodal/model_ckpts/Attention1DSTILRegressor exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
Restoring states from the checkpoint path at model_ckpts/Attention1DSTILRegressor/ckpt_best.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type            | Params
-----------------------------------------------
0 | attention  | Sequential      | 262 K 
1 | regressor  | Sequential      | 3.1 K 
2 | loss       | MSELoss         | 0     
3 | train_corr | PearsonCorrCoef | 0     
4 | train_r2   | R2Score         | 0     
5 | val_corr   | PearsonCorrCoef | 0     
6 | val_r2     | R2Score         | 0     
7 | test_corr  | Pea

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Monitored metric val_loss did not improve in the last 5 records. Best score: 0.015. Signaling Trainer to stop.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


training on device: cpu


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃   Runningstage.testing    ┃                           ┃
┃          metric           ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_corr         │    0.6733160614967346     │
│      test_corr_epoch      │    0.6733160614967346     │
│         test_loss         │     0.032563466578722     │
│          test_r2          │    0.2976086139678955     │
│       test_r2_epoch       │    0.2976086139678955     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.032563466578722,
  'test_corr': 0.6733160614967346,
  'test_r2': 0.2976086139678955,
  'test_r2_epoch': 0.2976086139678955,
  'test_corr_epoch': 0.6733160614967346}]